<img src="https://drive.google.com/uc?id=181S0KcAdGeAajZ1apcbOvoh3wvYuKtzd">
<b>Autores</b>:

*   Aldo von Wangenheim (aldo.vw@ufsc.br)
*   Rodrigo de Paula e Silva Ribeiro (ribeiro.rodrigo@posgrad.ufsc.br)


CERTIFIQUE-SE DE TER INCIADO O AMBIENTE EM MODO <b>GPU</b> ANTES DE REALIZAR OS PROCEDIMENTOS.<br>
NA TROCA DO AMBIENTE ELE <b><u>PERDE</u></b> TODOS OS DADOS BAIXADOS.

AO RE-EXECUTAR OS PROCEDIMENTOS (ex. diminuir o batch size), <br>REINICIE O AMBIENTE PARA LIBERAR O CACHE DO PYTORCH (reiniciar não perde os dados da sessão).<br>(alt+m ou pelo menu "Ambiente de Execução")

EXECUTE UTILIZANDO RUN ALL ou CTRL+F9

<a href="https://colab.research.google.com/github/awangenh/vision/blob/master/jupyter/14.3.Detecção%20de%20Sinais%20de%20Trânsito%20-%20YOLOv9.ipynb"><img align="left"  src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>&nbsp; &nbsp;<a href=""><img align="left" src="http://www.lapix.ufsc.br/wp-content/uploads/2019/04/License-CC-BY-ND-4.0-orange.png" alt="Creative Commons 4.0 License" title="Creative Commons 4.0 License"></a>&nbsp; &nbsp; <a href=""><img align="left" src="http://www.lapix.ufsc.br/wp-content/uploads/2019/04/Jupyter-Notebook-v.1.0-blue.png" alt="Jupyter Version" title="Jupyter Version"></a>&nbsp; &nbsp;<a href=""><img align="left"  src="https://img.shields.io/badge/python-3.10-greeng" alt="Python Version" title="Python Version"></a>

In [ ]:
!nvidia-smi
!nvcc --version

# Configurações de GPU e PATH

In [ ]:
!nvcc --version

try:
    import google.colab
    _ON_COLAB = True
except:
    _ON_COLAB = False
    %reload_ext autoreload
    %autoreload 2
    %matplotlib inline
    from IPython import get_ipython
    
print('Running on Google Colab = ', _ON_COLAB)
import os
import torch

# Definir o caminho base
if _ON_COLAB:
    BASE_PATH = "/content"
    import locale
    locale.getpreferredencoding = lambda: "UTF-8"
else:
    BASE_PATH = "/lapix" # trocar para o base path do seu computador se estiver rodando localmente, deixar /lapix se estiver rodando nos conteiners lapix
    VISIBLE_GPUS = [7] # Selecionar a GPU PARA RODAR! VER QUAL ESTA LIVRE

    if torch.cuda.device_count() != 8:
        print("GPU SETADA - PULANDO ETAPA")
    else:
        os.environ["CUDA_VISIBLE_DEVICES"] = str(VISIBLE_GPUS).replace("]", "").replace("[","").replace(" ", "")
        os.environ["NVIDIA_VISIBLE_DEVICES"] = str(VISIBLE_GPUS).replace("]", "").replace("[","").replace(" ", "")
        print("CUDA GPUS NUMBER: ", torch.cuda.device_count())
    
os.chdir(BASE_PATH) # garantir que está executando no caminho base definido

# Definir a pasta base atual.

In [ ]:
import os
HOME_DIR = os.getcwd()
print(HOME_DIR)

# O que é YOLO?
YOLO (“You Only Look Once”), é uma série de detectores de objetos de processamento rápido que vem sendo desenvolvida por diversos autores. Ao contrário dos detectores tradicionais que utilizam áreas de interesse para propor regiões (ROI) que podem conter um objeto na imagem (conhecidos como detectores de 2-estágios), a arquitetura YOLO divide a imagem em grids e vasculha a imagem como um todo (detector de um único estágio).

Neste notebook didático, é utilizado a versão 9 desta arquitetura.<br>
Algumas notas sobre a versão 9:<br>
* Em versões prévias era apenas possível utilizar detecção baseada em bounding box, desde a versão 7 já é possível utilizar também segmentação por instância e keypoint.
* Na versão 9, utiliza-se apenas o método OneCycle para otimização da Learning Rate, portanto não é necessário definir uma Learning Rate, a não ser que seja desejável utilizar Learning Rate fixa (linear).
* Foi conceitualizada pelos mesmos autores da YoloV4, Scaled-YoloV4 e YoloR. De fato é uma versão baseada na Scaled-YoloV4 com muitas melhorias.
* Diferente de outras versões que os pesos eram pré-treinados no conjunto imagenet, a versão 9, assim como a Scaled YoloV4, é pré-treinada no [COCO Dataset](https://cocodataset.org/#home).

Diferentemente do Detectron2 (framework customizável), YOLO é uma arquitetura parametrizada e única, embora é possível alterar diversos parâmetros no arquivo train.py oficial, muitas alterações ou valores muito diferentes podem descaracterizar a rede perdendo a sua capacidade de atingir resultados SOTA (state of the art).

Neste notebook apenas alguns parâmetros de performance são editáveis, para editar parâmetros mais específicos ou até mesmo implementar modificações, é recomendável ler e se familiarizar com a arquitetura.

Links:<br>
[Link Artigo](https://arxiv.org/abs/2402.13616) <br>
[Link git](https://github.com/WongKinYiu/yolov9)


# Instalação do YoloV9 do repositório oficial e suas dependências.

In [ ]:
!git clone https://github.com/WongKinYiu/yolov9.git
os.chdir(os.path.join(HOME_DIR, "yolov9"))
!pip install -r requirements.txt

# Montar o google drive
Se estiver rodando no Colab e quiser salvar no drive, execute a celula abaixo para montar o seu google drive e defina a pasta que será salva no Bloco de Parametros.

In [ ]:
if _ON_COLAB:
    from google.colab import drive
    drive.mount(os.path.join(HOME_DIR, 'gdrive'), force_remount=False)

# Baixar o dataset de itens de trânsito já no formato YOLO
O dataset de itens de trânsito tem como objetivo didático demonstrar a possibilidade de identificar alguns itens comuns de trânsito.<br>
É um conjunto anotado no formato "bounding box" para detecção de objetos.<br>
O conjunto possui 4 classes distintas:
* trafficlight
* stop
* speedlimit
* crosswalk

Amostra de anotações:<br>
<img src="https://drive.google.com/uc?id=1AoC87UUaJ5v3W03mTE6gQocW6sYgCEuz">


In [ ]:
import os
try:
  import gdown
  print("gdown já está instalado")
except ModuleNotFoundError:
  !python -m pip install gdown
os.chdir(HOME_DIR)
!gdown 1sIhCtoOMnpCXlByxDV95iRQDWjTPKr9v
!unzip -qq -u trafficsign.zip
!rm -rf trafficsign.zip


# Baixar o peso do modelo padrão para transfer learning.
Para mais opções de pesos pré-treinados e modelos maiores/menores, ver o github oficial do [YoloV9](https://github.com/WongKinYiu/yolov9).

In [ ]:
os.chdir(os.path.join(HOME_DIR, "yolov9"))
!wget https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt

# Parâmetros básicos
Implementação de edição dos parâmetros básicos de performance do Yolo v9.<br>
Para parâmetros opcionais e outros, ver [este código do git (linhas 438 à 487)](https://github.com/WongKinYiu/yolov9/blob/main/train_dual.py) e adequar o código.<br>



In [ ]:
# Definir onde salvar os resultados de treino
#SAVE_PATH = "{HOME_DIR}/gdrive/MyDrive/aula_yoloV9/" # para salvar na pasta 'aula_yoloV9' do seu drive, descomente esta linha e comente a linha abaixo.
SAVE_PATH = os.path.join(HOME_DIR, "yolov9_run") # salvar no ambiente local

os.makedirs(SAVE_PATH, exist_ok=True)

# Definir o batch size
BATCH_SIZE = 8

# Definir o número de épocas de treino
EPOCHS = 10

# Definir o formato de imagem para rede
IM_SIZE = 640 # padrão 640, imagem menor = mais rápido, menos preciso em objetos pequenos; imagem maior = mais lento, mais preciso em objetos pequenos.

In [ ]:
os.chdir(os.path.join(HOME_DIR, "yolov9"))
!python train_dual.py --device 0 --batch {BATCH_SIZE} --data "{HOME_DIR}/trafficsign/data.yaml" --img {IM_SIZE} --cfg models/detect/yolov9-c.yaml --weights 'yolov9-c.pt' --hyp hyp.scratch-high.yaml --min-items 0 --epochs {EPOCHS} --close-mosaic 15 --project "{SAVE_PATH}"

# Gráficos de treino

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from glob import glob
import matplotlib.ticker as ticker

metrics_img = glob(os.path.join(SAVE_PATH, 'exp/*.png'))
n_img = len(metrics_img)
n_cols = 2
n_rows = int(n_img / n_cols) + (1 if n_img % n_cols != 0 else 0)
_, axs = plt.subplots(n_rows, n_cols, figsize=(15, 15))
axs = axs.flatten()
for m, ax in zip(metrics_img, axs):
  img = plt.imread(m)
  ax.imshow(img)
  ax.xaxis.set_major_locator(ticker.NullLocator())
  ax.yaxis.set_major_locator(ticker.NullLocator())
plt.show()

# Realizar inferência de métricas no conjunto de validação (val/valid)

In [ ]:
curr_weight = os.path.join(SAVE_PATH, "exp/weights/best.pt") # trocar exp por exp1/exp2/exp3 conforme o número do último treino gerado. É incrementado automaticamente.
!python val_dual.py --data "{HOME_DIR}/trafficsign/data.yaml" --batch {BATCH_SIZE} --iou-thres 0.5 --device 0 --weights "{curr_weight}" --project "{SAVE_PATH}" --name "val"

# Visualização dos gráficos das métricas de validação

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from glob import glob
import matplotlib.ticker as ticker

metrics_img = glob(os.path.join(SAVE_PATH, "val/*.png"))
n_img = len(metrics_img)
n_cols = 2
n_rows = int(n_img / n_cols) + (1 if n_img % n_cols != 0 else 0)
_, axs = plt.subplots(n_rows, n_cols, figsize=(15, 15))
axs = axs.flatten()
for m, ax in zip(metrics_img, axs):
  img = plt.imread(m)
  ax.imshow(img)
  ax.xaxis.set_major_locator(ticker.NullLocator())
  ax.yaxis.set_major_locator(ticker.NullLocator())
plt.show()


# Visualizar detecções no conjunto de teste (test)

In [ ]:
curr_weight = os.path.join(SAVE_PATH, "exp/weights/best.pt") # trocar exp por exp1/exp2/exp3 conforme o número do último treino gerado. É incrementado automaticamente.
!python detect_dual.py --weights "{curr_weight}" --conf-thres 0.7 --source "{HOME_DIR}/trafficsign/images/test" --project "{SAVE_PATH}" --name "detect"

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from glob import glob
import matplotlib.ticker as ticker
import random

num_samples = 4 # numero de imagens para visualizar.
all_dets = glob(os.path.join(SAVE_PATH, "detect/*.png"))
n_cols = 2
n_rows = int(num_samples / n_cols) + (1 if num_samples % n_cols != 0 else 0)
_, axs = plt.subplots(n_rows, n_cols, figsize=(15, 15))
axs = axs.flatten()
for m, ax in zip(random.sample(all_dets, num_samples), axs):
  img = plt.imread(m)
  ax.imshow(img)
  ax.xaxis.set_major_locator(ticker.NullLocator())
  ax.yaxis.set_major_locator(ticker.NullLocator())
plt.show()

<img src="http://lapix.ufsc.br/wp-content/uploads/2022/10/rodape-lapix.png">